In [65]:
import pandas as pd
import numpy as np
from polygon import RESTClient
from polygon.rest.models.indicators import (
    SMAIndicatorResults,
    EMAIndicatorResults,
    RSIIndicatorResults,
    MACDIndicatorResults,
)

# Creating the list of s&p100 tickets

In [71]:
df = pd.read_csv('snp100.csv', header=None)

In [72]:
snp100tickers = df[0]

# Create downloading function

In [77]:
client = RESTClient(DATA_KEY)

def download_data(from_date, to_date, ticker, key):
    '''Downloads trading data for each ticker for the giving
    period in format Y-M-D and returns the Dataframe in the csv format'''
    client = RESTClient(f"{key}")
    aggs = []
    for a in client.list_aggs(
        f"{ticker}",
        1,
        "day",
        f"{from_date}",
        f"{to_date}",
        limit=5000,
        ):
        aggs.append(a)
# creating the basic dataframe
    data = pd.DataFrame(aggs)
# adding the columns with new features
# 25 days simple moving average
    sma25 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=25,
        series_type='close',
        limit=5000,
        )
    data['sma25'] = pd.DataFrame(sma25.values)['value']
# 100 days simple moving average    
    sma100 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=100,
        series_type='close',
        limit=5000,
        )
    data['sma100'] = pd.DataFrame(sma100.values)['value']
 # 200 days simple moving average    
    sma200 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=200,
        series_type='close',
        limit=5000,
        )
    data['sma200'] = pd.DataFrame(sma200.values)['value']
 # 14 days relative strength index    
    rsi = client.get_rsi(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=14,
        series_type='close',
        limit=5000,
        )
    data['rsi'] = pd.DataFrame(rsi.values)['value']
# moving average convergence/divergence
    macd = client.get_macd(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        short_window=12,
        long_window=26,
        signal_window=9,
        series_type='close',
        limit=5000,
        )
    data['macd'] = pd.DataFrame(macd.values)['value']
# adding volatility column
    # avg_price = np.sum(data['close'])/len(data['close'])
    # data['volatility'] = np.sqrt(np.sum(data['close']-avg_price**2)/(len(data['close'])-1))   
    return data.to_csv(f'raw_data/{ticker}.csv')

In [69]:
key= DATA_KEY

In [78]:
for ticker in snp100tickers:
    download_data('2018-08-28', '2023-08-28', ticker, key=key)